In [74]:
import numpy as np
import pandas as pd
import re

In [75]:
'''
Observaciones:
    1. Como en el enunciado se postula el problema enfocado en transporte público, se omitiran analisis
       respecto a usuarios que utilicen o dispongan de vehiculo privado/propio.
    2. A priori, como primer criterio para determinar viajes similares, se buscará informacion relacionada
       a la distancia media entre las minimas distancias entre barrios residenciales y sus paraderos de
       transporte publico mas cercanos.
    3. Dada que la hora es un marco de referencia 'absoluto' para fines de este estudio, mientras mas
       cercanos entre ellos sean (temporalmente hablando) hay mas posibilidades de que se traten de viajes
       similares.
    4. Dificultad a primera vista: ¿como asignar alguna especie de ponderaciones/pesos o criterios con el
       objetivo de determinar algun "score" de similitud?
'''

'\nObservaciones:\n    1. Como en el enunciado se postula el problema enfocado en transporte público, se omitiran analisis\n       respecto a usuarios que utilicen o dispongan de vehiculo privado/propio.\n    2. A priori, como primer criterio para determinar viajes similares, se buscará informacion relacionada\n       a la distancia media entre las minimas distancias entre barrios residenciales y sus paraderos de\n       transporte publico mas cercanos.\n    3. Dada que la hora es un marco de referencia \'absoluto\' para fines de este estudio, mientras mas\n       cercanos entre ellos sean (temporalmente hablando) hay mas posibilidades de que se traten de viajes\n       similares.\n    4. Dificultad a primera vista: ¿como asignar alguna especie de ponderaciones/pesos o criterios con el\n       objetivo de determinar algun "score" de similitud?\n'

In [76]:
def update_columns(func:str, col_name:str, col_list:list):
    if func is 'add' and col_name not in col_list:
        col_list.append(col_name)
    elif func is 'del' and col_name in col_list:
        col_list.remove(col_name)

In [77]:
df_raw_data = pd.read_csv('trips.csv')
columns_to_show = list(df_raw_data.columns.copy())
update_columns('del', 'datasource', columns_to_show)
df_raw_data.head()[columns_to_show]

,region,origin_coord,destination_coord,datetime
0,Prague,POINT (14.4973794438195 50.00136875782316),POINT (14.43109483523328 50.04052930943246),2018-05-28 09:03:40
1,Turin,POINT (7.672837913286881 44.9957109242058),POINT (7.720368637535126 45.06782385393849),2018-05-21 02:54:04
2,Prague,POINT (14.32427345662177 50.00002074358429),POINT (14.47767895969969 50.09339790740321),2018-05-13 08:52:25
3,Turin,POINT (7.541509189114433 45.09160503827746),POINT (7.74528653441973 45.02628598341506),2018-05-06 09:49:16
4,Turin,POINT (7.614078119815749 45.13433106465422),POINT (7.527497142312585 45.03335051325654),2018-05-23 12:45:54


In [78]:
#Preprocesamiento y filtrado de columnas que no seran utilizadas
#Separacion de los puntos en sus componentes (probable criterio para identificar cercania)
df_raw_data['origin_x'] = [float(re.findall(r'\d{1,2}\.\d*', x)[0]) for x in df_raw_data['origin_coord']]
df_raw_data['origin_y'] = [float(re.findall(r'\d{1,2}\.\d*', x)[1]) for x in df_raw_data['origin_coord']]
update_columns('del', 'origin_coord', columns_to_show)
update_columns('add', 'origin_x', columns_to_show)
update_columns('add', 'origin_y', columns_to_show)

df_raw_data['dest_x'] = [float(re.findall(r'\d{1,2}\.\d*', x)[0]) for x in df_raw_data['destination_coord']]
df_raw_data['dest_y'] = [float(re.findall(r'\d{1,2}\.\d*', x)[1]) for x in df_raw_data['destination_coord']]
update_columns('del', 'destination_coord', columns_to_show)
update_columns('add', 'dest_x', columns_to_show)
update_columns('add', 'dest_y', columns_to_show)

df_raw_data.head()[columns_to_show]

,region,datetime,origin_x,origin_y,dest_x,dest_y
0,Prague,2018-05-28 09:03:40,14.497379,50.001369,14.431095,50.040529
1,Turin,2018-05-21 02:54:04,7.672838,44.995711,7.720369,45.067824
2,Prague,2018-05-13 08:52:25,14.324273,50.000021,14.477679,50.093398
3,Turin,2018-05-06 09:49:16,7.541509,45.091605,7.745287,45.026286
4,Turin,2018-05-23 12:45:54,7.614078,45.134331,7.527497,45.033351


In [79]:
#Primera agrupacion trivial, por region
df_raw_data.groupby('region')['region'].describe()

,count,unique,top,freq
region,,,,
Hamburg,28,1,Hamburg,28
Prague,34,1,Prague,34
Turin,38,1,Turin,38


In [80]:
for reg in df_raw_data['region'].unique():
    mask = df_raw_data['region'] == str(reg)
    print(reg)
    print(df_raw_data[mask][columns_to_show[1:]])

Prague
               datetime   origin_x   origin_y     dest_x     dest_y
0   2018-05-28 09:03:40  14.497379  50.001369  14.431095  50.040529
2   2018-05-13 08:52:25  14.324273  50.000021  14.477679  50.093398
13  2018-05-20 02:31:22  14.343947  50.122997  14.450470  50.100777
15  2018-05-01 11:50:58  14.492907  50.016440  14.487357  50.027545
17  2018-05-06 12:56:14  14.419866  50.020125  14.410149  50.085190
21  2018-05-11 21:16:54  14.425596  50.053089  14.547629  50.005739
26  2018-05-08 04:22:01  14.658486  50.115713  14.612965  50.063478
27  2018-05-21 06:52:19  14.351462  50.103867  14.358828  50.004288
29  2018-05-02 13:48:30  14.425143  50.086267  14.405851  50.093318
32  2018-05-06 05:13:16  14.557828  50.084267  14.602243  50.107947
34  2018-05-12 15:12:04  14.568017  49.989806  14.605448  50.117497
35  2018-05-19 23:08:09  14.318404  50.084668  14.557625  50.085024
36  2018-05-03 01:35:44  14.666893  50.072040  14.452815  50.037519
40  2018-05-29 12:44:02  14.424060  50.07

In [81]:
#con fines practicos analizaremos Praga, teniendo en cuenta 
#que lo realizado debe funcionar para cualquier region
mask = df_raw_data['region'] == 'Prague'
df_prague = df_raw_data[mask]

In [85]:
#Se buscara construir una bounding box de los viajes dentro de la region para sectorizarlos
#en sub-regiones
min_x = df_prague['origin_x'].min() if df_prague['origin_x'].min() < df_prague['dest_x'].min() else df_prague['dest_x'].min()
max_x = df_prague['origin_x'].max() if df_prague['origin_x'].max() > df_prague['dest_x'].max() else df_prague['dest_x'].max()
min_y = df_prague['origin_y'].min() if df_prague['origin_y'].min() < df_prague['dest_y'].min() else df_prague['dest_y'].min()
max_y = df_prague['origin_y'].max() if df_prague['origin_y'].max() > df_prague['dest_y'].max() else df_prague['dest_y'].max()
# dif_origin_x = df_prague['origin_x'].max() - df_prague['origin_x'].min()
# dif_origin_y = df_prague['origin_y'].max() - df_prague['origin_y'].min()
# dif_dest_x =df_prague['dest_x'].max() - df_prague['dest_x'].min()
# dif_dest_y = df_prague['dest_y'].max() - df_prague['dest_y'].min()
# print(dif_origin_x,dif_origin_y)
# print(dif_dest_x,dif_dest_y)
print(min_x, max_x)
print(min_y, max_y)

14.31199296863995 14.66689257396927
49.98980557466577 50.12502233477432
